In [ ]:
%matplotlib inline
import numpy as np
import scipy.spatial
import pandas as pd
import sklearn.decomposition
import matplotlib.pyplot as plt
import seaborn as sb

import linear_cca
import multimodal_data

# Useful References
## https://arxiv.org/pdf/1711.02391.pdf
## http://users.stat.umn.edu/~helwig/notes/cancor-Notes.pdf
## https://www.statisticssolutions.com/canonical-correlation/

# Load data

In [ ]:
l1k = multimodal_data.load_l1000("treatment_level_all_alleles.csv")
l1k = multimodal_data.load_l1000("replicate_level_all_alleles.csv")
cp = multimodal_data.load_cell_painting(
    "/data1/luad/others/morphology.csv", 
    "resnet18-validation-well_profiles.csv", 
    aggregate_replicates=False
)

In [ ]:
l1k, cp = multimodal_data.align_profiles(l1k, cp, sample=4)
common_alleles = set(cp["Allele"].unique()).intersection( l1k["Allele"].unique() )
genes = list(common_alleles)
genes = [x for x in genes if x not in ["EGFP", "BFP", "HCRED"]]
l1k = l1k[l1k.Allele.isin(genes)]
cp = cp[cp.Allele.isin(genes)]

# Compute CCA

In [ ]:
# Preprocessing to the data:
# 1. Standardize features (z-scoring)
# 2. Reduce dimensionality (PCA down to 100 features)
# This is necessary because we only have 175 data points, 
# while L1000 has 978 features and Cell Painting has 256.
# So PCA is useful as a regularizer somehow.

def cca_analysis(GE_train, MF_train, GE_test, MF_test):
    # Prepare Gene Expression matrix
    sc_l1k = sklearn.preprocessing.StandardScaler()
    sc_l1k.fit(GE_train)
    GE = sc_l1k.transform(GE_train)
    
    pca_l1k = sklearn.decomposition.PCA(n_components=150, svd_solver="full")
    pca_l1k.fit(GE)
    GE = pca_l1k.transform(GE)

    # Prepare Cell Painting matrix
    sc_cp = sklearn.preprocessing.StandardScaler()
    sc_cp.fit(MF_train)
    MF = sc_cp.transform(MF_train)
    
    pca_cp = sklearn.decomposition.PCA(n_components=100, svd_solver="full")
    pca_cp.fit(MF)
    MF = pca_cp.transform(MF)

    # Compute CCA
    A, B, D, ma, mb = linear_cca.linear_cca(MF, GE, 10)
    
    X = pca_cp.transform(sc_cp.transform(MF_test))
    Y = pca_l1k.transform(sc_l1k.transform(GE_test))
    
    X = np.dot(X, A)
    Y = np.dot(Y, B)
    
    return X, Y, D

In [ ]:
GE = np.asarray(l1k)[:,1:]
MF = np.asarray(cp)[:,1:]
MF_v, GE_v, D = cca_analysis(GE, MF, GE, MF)

In [ ]:
# In linear CCA, the canonical correlations equal to the square roots of the eigenvalues:
plt.plot(np.sqrt(D))
print("First cannonical correlation: ", np.sqrt(D[0]))

In [ ]:
D = scipy.spatial.distance_matrix(MF_v[:,0:2], GE_v[:,0:2])
NN = np.argsort(D, axis=1) # Nearest morphology point to each gene expression point

plt.figure(figsize=(10,10))
plt.scatter(MF_v[:,0], MF_v[:,1], c="blue", s=50, edgecolor='gray', linewidths=1)
plt.scatter(GE_v[:,0]+0, GE_v[:,1]+0, c="lime", edgecolor='gray', linewidths=1)

connected = 0
for i in range(MF_v.shape[0]):
    for j in range(7): #GE_v.shape[0]):
        if cp.iloc[i].Allele == l1k.iloc[NN[i,j]].Allele:
            plt.plot([GE_v[NN[i,j],0],MF_v[i,0]],[GE_v[NN[i,j],1],MF_v[i,1]], 'k-', color="red")
#             if np.random.random() > 0.9:
#                 plt.text(GE_v[i,0], GE_v[i,1], l1k.iloc[i].Allele, horizontalalignment='left', size='medium', color='black')
            connected += 1
            #break

print(connected)
# plt.xlim(-2,2)
# plt.ylim(-2,2)

In [ ]:
df = pd.DataFrame(data={"cca1": np.concatenate((GE_v[:,0], MF_v[:,0])), 
                   "cca2": np.concatenate((GE_v[:,1],MF_v[:,1])),
                   "source": ["L1K" for x in range(GE_v.shape[0])]+["CP" for x in range(MF_v.shape[0])],
                   "allele": list(l1k["Allele"]) + list(cp["Allele"])}
                 )

In [ ]:
df["color"] = df["allele"].str.find("EGFR") != -1
sb.lmplot(data=df, x="cca1", y="cca2", hue="color", fit_reg=False, col="source")

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(MF_v[:,0], MF_v[:,1], c="blue", s=100, edgecolor='gray', linewidths=1)
plt.figure(figsize=(10,10))
plt.scatter(GE_v[:,0]+0, GE_v[:,1]+0, c="lime", s=100, edgecolor='gray', linewidths=1)

# Annotate visualization

In [ ]:
def visualize_annotations(l1k, cp, GE_v, MF_v, display_items=[]):
    ge_data = pd.DataFrame(data=l1k["Allele"].reset_index()) 
    ge_data["x"] = GE_v[:,0]
    ge_data["y"] = GE_v[:,1]
    ge_data.columns = ["idx", "Allele", "x", "y"]
    ge_data["type"] = "GeneExpression"
    
    mf_data = pd.DataFrame(data=cp["Allele"].reset_index())
    mf_data["x"] = MF_v[:,0]
    mf_data["y"] = MF_v[:,1]
    mf_data.columns = ["idx", "Allele", "x", "y"]
    mf_data["type"] = "Morphology"
    
    data = pd.concat([ge_data, mf_data])

    plt.figure(figsize=(12,12))
    p1 = sb.regplot(data=ge_data, x="x", y="y", fit_reg=False, color="red", scatter_kws={'s':50})
    p2 = sb.regplot(data=mf_data, x="x", y="y", fit_reg=False, color="blue", scatter_kws={'s':50})

    for point in range(ge_data.shape[0]):
        #if ge_data.Allele[point] in display_items:
        p1.text(ge_data.x[point], ge_data.y[point], ge_data.Allele[point], horizontalalignment='left', size='medium', color='black')

    for point in range(mf_data.shape[0]):
        #if mf_data.Allele[point] in display_items:
        p2.text(mf_data.x[point], mf_data.y[point], mf_data.Allele[point], horizontalalignment='left', size='medium', color='black')

In [ ]:
visualize_annotations(l1k, cp, GE_v, MF_v, display_items=["NFE2L2_p.T80K","EGFP"])

# Visualization in the test set

In [ ]:
common_alleles = set(cp["Allele"].unique()).intersection( l1k["Allele"].unique() )
genes = list(common_alleles)
np.random.shuffle(genes)

train = genes[0:9*int(len(genes)/10)]
test = genes[9*int(len(genes)/10):]

GE_train = np.asarray(l1k[l1k["Allele"].isin(train)])[:,1:]
MF_train = np.asarray(cp[cp["Allele"].isin(train)])[:,1:]

GE_test = np.asarray(l1k[l1k["Allele"].isin(test)])[:,1:]
MF_test = np.asarray(cp[cp["Allele"].isin(test)])[:,1:]

MF_v, GE_v, D = cca_analysis(GE_train, MF_train, GE_test, MF_test)

visualize_annotations(
    l1k[l1k["Allele"].isin(test)], 
    cp[cp["Allele"].isin(test)], 
    GE_v, 
    MF_v
)

In [ ]:
D = scipy.spatial.distance_matrix(MF_v[:,0:2], GE_v[:,0:2])
NN = np.argsort(D, axis=1) # Nearest morphology point to each gene expression point

plt.figure(figsize=(10,10))
plt.scatter(MF_v[:,0], MF_v[:,1], c="blue", s=50, edgecolor='gray', linewidths=1)
plt.scatter(GE_v[:,0]+0, GE_v[:,1]+0, c="red", edgecolor='gray', linewidths=1)

connected = 0
for i in range(MF_v.shape[0]):
    for j in range(7):
        if cp.iloc[i].Allele == l1k.iloc[NN[i,j]].Allele:
            plt.plot([GE_v[NN[i,j],0],MF_v[i,0]],[GE_v[NN[i,j],1],MF_v[i,1]], 'k-', color="lime")

In [ ]:
# In linear CCA, the canonical correlations equal to the square roots of the eigenvalues:
plt.plot(np.sqrt(D))
print("First cannonical correlation: ", np.sqrt(D[0]))

# Visualize data matrices

In [ ]:
X = (GE - np.min(GE))/(np.max(GE) - np.min(GE))
X = np.asarray(X, dtype=np.float32)
plt.imshow(X)

In [ ]:
X = (MF - np.min(MF))/(np.max(MF) - np.min(MF))
X = np.asarray(X, dtype=np.float32)
plt.imshow(X)